In [35]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3 as sql

# SQL EXAM REVIEW

Thank you to sqlitetutorial for the database!

* https://www.sqlitetutorial.net/

There are 11 tables in the chinook sample database.  The description and the database schema are below.

* ```employees``` table stores employees data such as employee id, last name, first name, etc. It also has a field named ReportsTo to specify who reports to whom.
* ```customers``` table stores customers data.
*  ```invoices``` & ```invoice_items``` tables: these two tables store invoice data. The invoices table stores invoice header data and the invoice_items table stores the invoice line items data.
* ```artists``` table stores artists data. It is a simple table that contains only the artist id and name.
* ```albums``` table stores data about a list of tracks. Each album belongs to one artist. However, one artist may have multiple albums.
* ```media_types``` table stores media types such as MPEG audio and AAC audio files.
* ```genres``` table stores music types such as rock, jazz, metal, etc.
* ```tracks``` table stores the data of songs. Each track belongs to one album.
* ```playlists``` & ```playlist_track``` tables: ```playlists``` table store data about ```playlists```. Each ```playlist``` contains a list of tracks. Each track may belong to multiple playlists. The relationship between the ```playlists``` table and ```tracks``` table is many-to-many. The playlist_track table is used to reflect this relationship.

---

# There is an error in the schema - READ ME!

* The ```invoice_items``` table does not have an ```InvoiceItemId```.  Instead it has a ```InvoiceLineId```.

![Sample Database](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

---

In [36]:
try:
     
    # Making a connection between sqlite3
    # database and Python Program
    conn = sql.connect('chinook.db')
     
    # If sqlite3 makes a connection with python
    # program then it will print "Connected to SQLite"
    # Otherwise it will show errors
    print("Connected to SQLite")
 
    # Getting all tables from sqlite_master
    sql_query = """SELECT name FROM sqlite_master
    WHERE type='table';"""
 
except sqlite3.Error as error:
    print("Failed to execute the above query", error)

Connected to SQLite


# Q1

How many people have the job title IT staff?

In [37]:
sql_statement = """SELECT COUNT(Title) AS num_it_staff
                   FROM employees
                   WHERE Title = 'IT Staff'
                """

results = pd.read_sql_query(sql_statement, conn)
results

,num_it_staff
0,2


# Q2

Find the total number of customers per country in the database and show the results in alphabetical order by country.

In [38]:
sql_statement = """SELECT country, COUNT(*) AS num_customers
                   FROM customers
                   GROUP BY country
                   ORDER BY country ASC"""

results = pd.read_sql_query(sql_statement, conn)
results

,Country,num_customers
0,Argentina,1
1,Australia,1
2,Austria,1
3,Belgium,1
4,Brazil,5
5,Canada,8
6,Chile,1
7,Czech Republic,2
8,Denmark,1
9,Finland,1


# Q3

Show how many records are missing from each column in the customers table.

In [39]:
sql_statement = """SELECT COUNT(*)-COUNT(FirstName) AS num_missing_fn,
                    COUNT(*)-COUNT(LastName) AS num_missing_ln,
                    COUNT(*)-COUNT(Company) AS num_missing_co,
                    COUNT(*)-COUNT(Address) AS num_missing_add,
                    COUNT(*)-COUNT(City) AS num_missing_city,
                    COUNT(*)-COUNT(State) AS num_missing_st,
                    COUNT(*)-COUNT(PostalCode) AS num_missing_postal_code,
                    COUNT(*)-COUNT(Phone) AS num_missing_phone,
                    COUNT(*)-COUNT(Fax) AS num_missing_fax,
                    COUNT(*)-COUNT(Email) AS num_missing_email,
                    COUNT(*)-COUNT(SupportRepId) AS num_missing_support_rep
                   FROM customers"""

results = pd.read_sql_query(sql_statement, conn)
results

,num_missing_fn,num_missing_ln,num_missing_co,num_missing_add,num_missing_city,num_missing_st,num_missing_postal_code,num_missing_phone,num_missing_fax,num_missing_email,num_missing_support_rep
0,0,0,49,0,0,29,4,1,47,0,0


# Q4

Find the number of customers in each state of our current country of residence.   Show the results in alphabetical order by state.

In [40]:
sql_statement = """SELECT state, COUNT(*) AS num_customers
                   FROM customers
                   WHERE country = 'USA'
                   GROUP BY state
                   ORDER BY state ASC"""

results = pd.read_sql_query(sql_statement, conn)
results

,State,num_customers
0,AZ,1
1,CA,3
2,FL,1
3,IL,1
4,MA,1
5,NV,1
6,NY,1
7,TX,1
8,UT,1
9,WA,1


# Q5

Select all customers from France who ordered songs in the Opera genre.

## NOTE:  
* Opera gives 0 but if you switch to Metal in your query you should get 4 customers

In [41]:
sql_statement = """SELECT DISTINCT(c.CustomerId)
                   FROM   customers  AS c
                          JOIN 
                            invoices  AS i
                            USING(CustomerId)
                          JOIN  
                            invoice_items AS ii
                            USING(InvoiceId)
                   WHERE c.Country = 'France'  
                         AND ii.TrackId IN  (SELECT t.TrackId 
                                            FROM tracks As t
                                                 JOIN
                                                   genres AS g
                                                   USING(GenreId)
                                            WHERE g.Name Like 'Metal')
            """

results = pd.read_sql_query(sql_statement, conn)
results

,CustomerId
0,42
1,39
2,41
3,40


# Q6

Find the number of employees under each manager.

In [42]:
#First I did this query to see how many managers were in the table so I could check my results.
#There are 3 managers - they have employeeId 1, 2, 6
sql_statement = """SELECT DISTINCT(ReportsTo)
                   FROM   employees 
            """

results = pd.read_sql_query(sql_statement, conn)
results

,ReportsTo
0,NaN
1,1.0
2,2.0
3,6.0


In [43]:
#NOTE:
#Is it a problem that we mixed aggregated and non-aggregated cols below?
#ANSWER: NO! We groupped by managers.employeeId, and each manager has a single first/LastName 
#        so we don't need to worry about aggregating first/last name.

sql_statement = """SELECT managers.EmployeeId AS manager_id, 
                          managers.FirstName AS manager_FN,             
                          managers.LastName AS manager_LN, 
                          COUNT(minions.ReportsTo) AS num_employees_under
                   FROM   employees AS managers
                          JOIN
                            employees AS minions
                            ON (managers.EmployeeId = minions.ReportsTo)
                   GROUP BY managers.EmployeeId
            """

results = pd.read_sql_query(sql_statement, conn)
results

,manager_id,manager_FN,manager_LN,num_employees_under
0,1,Andrew,Adams,2
1,2,Nancy,Edwards,3
2,6,Michael,Mitchell,2


# Q7

How many last names that start with “S” are shown on the customer's table but not on the employees' table?

## NOTE:  

* There is only one letter (M) with a repeated last name of that letter between customers/employees. Challenge: How did I verify that?

* So 'S' gives 0 but if you switch to 'M' you'll get 6 even though there are 7 customers with an M-last name in the DB.  

In [44]:
sql_statement = """SELECT c.LastName 
                   FROM   customers AS c
                   WHERE c.LastName LIKE 'M%'
                   
                   EXCEPT 
                   
                   SELECT e.LastName 
                   FROM   employees AS e
                   WHERE  e.LastName LIKE 'M%'  --Don't actually need for a right answer.  Why?
            """

results = pd.read_sql_query(sql_statement, conn)
results

,LastName
0,Mancini
1,Martins
2,Mercier
3,Miller
4,Murray
5,Muñoz


# Q8

Count the number of employees born in each season and include a count for those employees whose season of birth is unknown.
* March-May is Spring
* June-August
* September-November is Fall
* December-February is Winter

## IMPORTANT NOTE:
* In reality, the answer is not a realistic way to do this problem but it is a GREAT EXAMPLE OF CASE-WHEN-THEN.  In the answer below, we manipul;ate the birthdates as if they are strings. Dates are actually not manipulated like strings in real world SQL.  Why should strings and dates be very different? Well, for one thing, we cannot subtract strings in SQL (ex:  'Ann' - 'HOUCK' causes an error as this subtraction is not sensible).  But we should be able to subtract dates:  01-04-2022 - 01-01-2022  = 3 days. So in real world SQL, dates are actually considerd to be special objects with special functions to process them.  

In [45]:
sql_statement = """SELECT CASE
                              WHEN BirthDate LIKE "%-01-%" THEN 'WINTER'
                              WHEN BirthDate LIKE "%-02-%" THEN 'WINTER'
                              WHEN BirthDate LIKE "%-03-%" THEN 'SPRING'
                              WHEN BirthDate LIKE "%-04-%" THEN 'SPRING' 
                              WHEN BirthDate LIKE "%-05-%" THEN 'SPRING'
                              WHEN BirthDate LIKE "%-06-%" THEN 'SUMMER'
                              WHEN BirthDate LIKE "%-07-%" THEN 'SUMMER' 
                              WHEN BirthDate LIKE "%-08-%" THEN 'SUMMER' 
                              WHEN BirthDate LIKE "%-09-%" THEN 'FALL' 
                              WHEN BirthDate LIKE "%-10-%" THEN 'FALL' 
                              WHEN BirthDate LIKE "%-11-%" THEN 'FALL' 
                              WHEN BirthDate LIKE "%-12-%" THEN 'WINTER' 
                              ELSE 'UNKNOWN'
                          END AS birth_season,
                          COUNT(EmployeeId) AS num_emps
                   FROM   employees
                   GROUP BY birth_season
                   
            """

results = pd.read_sql_query(sql_statement, conn)
results

,birth_season,num_emps
0,FALL,1
1,SPRING,2
2,SUMMER,2
3,WINTER,3


# Q9

How many more times does the country we reside in pay in total invoices compared to the UK?

In [46]:
sql_statement = """SELECT SUM(ii.UnitPrice * ii.Quantity)/ (SELECT SUM(ii2.UnitPrice * ii2.Quantity) 
                                                              FROM customers AS c2 
                                                                JOIN invoices AS i2 
                                                                     USING(CustomerId) 
                                                                JOIN invoice_items AS ii2
                                                                     USING (InvoiceId) 
                                                               WHERE c2.country = 'United Kingdom') AS USA_TO_UK_Total_Invoices
                   FROM customers AS c 
                        JOIN invoices AS i 
                             USING(CustomerId) 
                        JOIN invoice_items AS ii
                             USING (InvoiceId) 
                   WHERE c.country = 'USA'
            """

results = pd.read_sql_query(sql_statement, conn)
results

,USA_TO_UK_Total_Invoices
0,4.634592


# Q10

Show the artists that do not have any albums in the database by using: 
* One of Union, Union All, Intersect, or Except 
* An Anti-Join or Semi-Join (so NOT IN or IN)

Just show artistId.

In [47]:
#OPTION 1
sql_statement = """SELECT artistId FROM artists
                   EXCEPT
                   SELECT artistId FROM albums
                   
            """

results = pd.read_sql_query(sql_statement, conn)
results

,ArtistId
0,25
1,26
2,28
3,29
4,30
...,...
66,192
67,193
68,194
69,195


In [48]:
#OPTION 2
sql_statement = """SELECT artistId 
                   FROM artists 
                   WHERE artistId NOT IN (SELECT artistId 
                                          FROM albums)
            """

results = pd.read_sql_query(sql_statement, conn)
results

,ArtistId
0,25
1,26
2,28
3,29
4,30
...,...
66,192
67,193
68,194
69,195


# Q11

If this question/song, offends anyone, then I ask your forgiveness.  The choice was unintentional.  I should have looked up what the phrase means.

Select all albums that have a longer title than “Balls to the Walls”.  Show the album title and the number of letters in the title and also sort by the number of letters from most to highest.

In [49]:

sql_statement = """SELECT Title AS album_title, LENGTH(Title) AS num_letters
                   FROM albums 
                   WHERE LENGTH(Title) > LENGTH('Balls to the Walls')
                   ORDER BY num_letters DESC
            """

results = pd.read_sql_query(sql_statement, conn)
results

,album_title,num_letters
0,"Tchaikovsky: 1812 Festival Overture, Op.49, Ca...",95
1,"J.S. Bach: Chaconne, Suite in E Minor, Partita...",86
2,Great Performances - Barber's Adagio and Other...,77
3,"Pure Cult: The Best Of The Cult (For Rockers, ...",76
4,20th Century Masters - The Millennium Collecti...,71
...,...,...
179,Misplaced Childhood,19
180,Every Kind of Light,19
181,Sibelius: Finlandia,19
182,English Renaissance,19


# Q12

## NOTE/CHALLENGE:  
* I rephrased the question so you have to change units.

For each track, list its name and how many seconds longer/shorter (in seconds) it is compared to "Restless and Wild".

In [50]:

sql_statement = """SELECT Name, 
                          (Milliseconds - (Select Milliseconds FROM tracks where Name = 'Restless and Wild'))/1000.0 AS diff_from_rw_in_secs
                   FROM tracks
            """

results = pd.read_sql_query(sql_statement, conn)
results

,Name,diff_from_rw_in_secs
0,For Those About To Rock (We Salute You),91.668
1,Balls to the Wall,90.511
2,Fast As a Shark,-21.432
3,Restless and Wild,0.000
4,Princess of the Dawn,123.367
...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,34.690
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",-112.851
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",-185.412
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",-30.720


# Q13

Find out the percentage of tracks with bytes bigger than 5  million in the tracks table.  

## NOTE: 
* Since no record has a null bytes value, it does not matter whether we use ```COUNT(*)``` or ```COUNT(Bytes)``` in the denominator for our percentage.

In [51]:
#First - A Quick Check: are any bytes null?  ANSWER: No.  So we don't have to argue about the denominator of our percentage.

sql_statement = """SELECT COUNT(*)
                   FROM tracks
                   WHERE BYTES IS NULL
            """


results = pd.read_sql_query(sql_statement, conn)
results

,COUNT(*)
0,0


In [52]:

sql_statement = """SELECT 100.0 * COUNT(BYTES)/
                              (SELECT COUNT(*) FROM tracks) AS percent_above_5mil
                   FROM tracks
                   WHERE BYTES > 5000000
            """

results = pd.read_sql_query(sql_statement, conn)
results

,percent_above_5mil
0,87.69626


# Q14

Select albums featuring less than 10 songs.  Show the count of songs for the album and sort your results by the number of songs, MOST to LEAST.

## NOTE:
* Should use LEFT JOIN and not just an inner join here.  Why?  There could have been albums with no tracks in the database.  In that case, there'd be a row with "Title of Album 0" and 0 is less than 10.

In [53]:
#Okay to mix aggregated and non-aggregated columns here because each AlbumId has exactly one Title.

sql_statement = """SELECT a.AlbumId, a.Title AS album_title, COUNT(*)  AS num_songs
                   FROM albums AS a
                        LEFT JOIN tracks AS t 
                        USING(AlbumId) 
                   GROUP BY a.AlbumId
                   HAVING num_songs < 10
                   ORDER BY num_songs DESC
                   
            """

results = pd.read_sql_query(sql_statement, conn)
results

,AlbumId,album_title,num_songs
0,31,Bongo Fury,9
1,58,Come Taste The Band,9
2,64,Slaves And Masters,9
3,65,Stormbringer,9
4,68,Outbreak,9
...,...,...,...
132,343,Respighi:Pines of Rome,1
133,344,Schubert: The Late String Quartets & String Qu...,1
134,345,Monteverdi: L'Orfeo,1
135,346,Mozart: Chamber Music,1


# Q15

Find the names of the tracks that are MPEG-4 file types.

In [54]:
sql_statement = """SELECT t.Name
                   FROM tracks  AS t
                         JOIN
                        media_types AS m
                        USING(MediaTypeId)
                   WHERE m.Name LIKE '%MPEG-4%'
            """

results = pd.read_sql_query(sql_statement, conn)
results

,Name
0,Battlestar Galactica: The Story So Far
1,Occupation / Precipice
2,"Exodus, Pt. 1"
3,"Exodus, Pt. 2"
4,Collaborators
...,...
209,"There's No Place Like Home, Pt. 2"
210,"There's No Place Like Home, Pt. 3"
211,"Band Members Discuss Tracks from ""Revelations"""
212,Branch Closing


# Q16

Which tracks, if any, have not been ordered through any invoices?

In [55]:
#Option 1
sql_statement = """SELECT TrackId, Name
                   FROM tracks
                   WHERE TrackId NOT IN (SELECT TrackId 
                                         FROM invoice_items)
            """

results = pd.read_sql_query(sql_statement, conn)
results

,TrackId,Name
0,7,Let's Get It Up
1,11,C.O.D.
2,17,Let There Be Rock
3,18,Bad Boy Boogie
4,22,Whole Lotta Rosie
...,...,...
1514,3497,"Erlkonig, D.328"
1515,3498,"Concerto for Violin, Strings and Continuo in G..."
1516,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)"
1517,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ..."


In [56]:
#Option 2 - Can't as easily select the name here 
sql_statement = """SELECT TrackId
                   FROM tracks
                   
                   EXCEPT
                   
                   SELECT TrackId 
                   FROM invoice_items
            """

results = pd.read_sql_query(sql_statement, conn)
results

,TrackId
0,7
1,11
2,17
3,18
4,22
...,...
1514,3497
1515,3498
1516,3501
1517,3502


# Q17

Show any city that is the hometown of an employee or customer. Order them in alphabetical order.


In [57]:
sql_statement = """ Select City 
                    FROM customers
                    
                    UNION
                    
                    Select City 
                    FROM employees
                    
                    ORDER BY City
            """

results = pd.read_sql_query(sql_statement, conn)
results

,City
0,Amsterdam
1,Bangalore
2,Berlin
3,Bordeaux
4,Boston
5,Brasília
6,Brussels
7,Budapest
8,Buenos Aires
9,Calgary


# Q18

Give the name of track with the largest file size in the tracks table .

## NOTE:  THE WRONG WAY
* This is a lot like the HW 5 mistake we discussed.  So the following query would be an INCORRECT way to do this, in case there are multiple records with the max size.

```SELECT Name, MAX(Bytes)```

```FROM tracks```

In [58]:
#correct way
sql_statement = """SELECT Name, Bytes
                   FROM tracks
                   Where Bytes = (SELECT MAX(Bytes) 
                                  From tracks)
            """

results = pd.read_sql_query(sql_statement, conn)
results

,Name,Bytes
0,Through a Looking Glass,1059546140


# Q19

What is the unit price of the track "Fast as a Shark"?

Note:  The 'as' is capitalized in the DB.

In [59]:
sql_statement = """SELECT Name, UnitPrice 
                   FROM tracks 
                   WHERE Name Like '%Shark%'   -- Could also do: WHERE Name = 'Fast As a Shark'
            """

results = pd.read_sql_query(sql_statement, conn)
results

,Name,UnitPrice
0,Fast As a Shark,0.99


# Q20

List any albums with mixed genres.

In [60]:

sql_statement = """SELECT Albumid,
                          Title AS album_title, 
                          COUNT(Distinct(Genreid)) AS num_genres
                   FROM genres AS g
                          JOIN tracks AS t      
                             USING(Genreid)
                          JOIN albums AS al
                             USING(AlbumId)
                   GROUP BY Albumid
                   HAVING num_genres > 1
                   ORDER BY num_genres DESC"""

results = pd.read_sql_query(sql_statement, conn)
results



,AlbumId,album_title,num_genres
0,141,Greatest Hits,3
1,227,"Battlestar Galactica, Season 3",3
2,73,Unplugged,2
3,102,Live After Death,2
4,109,Rock In Rio [CD2],2
5,112,The Number of The Beast,2
6,228,"Heroes, Season 1",2
7,229,"Lost, Season 3",2
8,231,"Lost, Season 2",2
9,251,"The Office, Season 3",2


# Q21

List the artist against the number of genres they have written. Order the artists alphabetically.

In [61]:

sql_statement = """SELECT ar.Name AS artist_name, 
                          COUNT(Distinct(Genreid)) AS num_genres
                   FROM genres AS g
                          JOIN tracks AS t       --For the genre
                             USING (Genreid)
                          JOIN albums AS al      --For the album
                             USING(AlbumId)
                          JOIN artists as ar     --For the artist
                             USING(ArtistId)
                   GROUP BY ar.Name
                   HAVING num_genres > 1
                   ORDER BY ar.Name ASC"""

results = pd.read_sql_query(sql_statement, conn)
results



,artist_name,num_genres
0,Amy Winehouse,2
1,Antônio Carlos Jobim,2
2,Audioslave,3
3,Battlestar Galactica,3
4,Eric Clapton,2
5,Faith No More,2
6,Foo Fighters,2
7,Gilberto Gil,3
8,Guns N' Roses,2
9,Heroes,2


# Q22

Find the 3 artists with the most albums in the database (ordered from most to least).  Show their id, name, and number of albums.



In [62]:
sql_statement = """SELECT ArtistId, Name, COUNT(*) as num_albums
                   FROM artists LEFT JOIN albums USING(ArtistId)
                   GROUP BY ArtistId
                   ORDER BY Count(*) DESC
                   LIMIT 3
                   """

results = pd.read_sql_query(sql_statement, conn)
results

,ArtistId,Name,num_albums
0,90,Iron Maiden,21
1,22,Led Zeppelin,14
2,58,Deep Purple,11


# Q23

List the different media_types (without repeats) in the database.

In [63]:
sql_statement = """SELECT DISTINCT(Name) 
                   FROM media_types
                   ORDER BY Name
                   """

results = pd.read_sql_query(sql_statement, conn)
results

,Name
0,AAC audio file
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file


# Q24

Show how many tracks per playlist, including  any playlists that have no tracks.

In [64]:
sql_statement = """SELECT Name, PlaylistId, COUNT(TrackId)
                   FROM playlists LEFT JOIN playlist_track USING(PlaylistId)
                   GROUP BY PlaylistId
                   ORDER BY Name ASC
                   """

results = pd.read_sql_query(sql_statement, conn)
results

,Name,PlaylistId,COUNT(TrackId)
0,90’s Music,5,1477
1,Audiobooks,4,0
2,Audiobooks,6,0
3,Brazilian Music,11,39
4,Classical,12,75
5,Classical 101 - Deep Cuts,13,25
6,Classical 101 - Next Steps,14,25
7,Classical 101 - The Basics,15,25
8,Grunge,16,15
9,Heavy Metal Classic,17,26


# Q25

Notice there are 2 playlists for TV Shows.  

Count how many songs are common to both lists using:

* A) One of UNION, UNION ALL, INTERSECT, or EXCEPT. 
* B) A Semi-Join (so a subquery)


In [65]:
sql_statement = """SELECT DISTINCT(t.trackId)
                   FROM playlists as pl LEFT JOIN playlist_track AS plt USING(PlaylistId) 
                           JOIN tracks as t USING(TrackId) 
                   WHERE plt.PlaylistId IN (13, 14, 15)
                   ORDER BY t.Name ASC
                   """

results = pd.read_sql_query(sql_statement, conn)
results

,TrackId
0,3412
1,3495
2,3487
3,3481
4,3486
...,...
70,3430
71,3424
72,3441
73,3442


In [66]:
sql_statement = """SELECT t.Name, PlaylistId
                   FROM playlists as pl LEFT JOIN playlist_track AS plt USING(PlaylistId) 
                           JOIN tracks as t USING(TrackId) 
                   WHERE plt.PlaylistId IN (1, 8)
                   ORDER BY t.Name ASC
                   """

results = pd.read_sql_query(sql_statement, conn)
results

,Name,PlaylistId
0,"""40""",1
1,"""40""",8
2,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525...",1
3,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525...",8
4,#1 Zero,1
...,...,...
6575,Óculos,8
6576,Óia Eu Aqui De Novo,1
6577,Óia Eu Aqui De Novo,8
6578,Último Pau-De-Arara,1


# Q26

Show how many times each song appears in a playlist, showing any songs that have no tracks.

In [67]:
sql_statement = """SELECT t.Name, COUNT(t.TrackId) as num_playlists
                   FROM tracks AS t LEFT JOIN playlist_track AS plt USING (TrackId) 
                   GROUP BY t.TrackId
                   ORDER BY num_playlists DESC
                   """

results = pd.read_sql_query(sql_statement, conn)
results

,Name,num_playlists
0,Intoitus: Adorate Deum,5
1,"Miserere mei, Deus",5
2,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg V...",5
3,"Suite for Solo Cello No. 1 in G Major, BWV 100...",5
4,"The Messiah: Behold, I Tell You a Mystery... T...",5
...,...,...
3498,October Song,2
3499,What Is It About Men,2
3500,Help Yourself,2
3501,Amy Amy Amy (Outro),2


# Q27 - CHALLENGING

Find the min/max/average quantity of items and the min/max/average invoice total for those who live in and out of the country we currently reside in.

In [68]:
sql_statement = """SELECT  CASE 
                                WHEN c.country = 'USA' THEN 'USA'
                                WHEN c.country != 'USA' THEN 'Other'
                                ELSE 'No country listed'
                            END AS usa_or_not, 
                          MIN(num_invoice_items) AS min_num_invoice_items, 
                          MAX(num_invoice_items) AS max_num_invoice_items, 
                          AVG(num_invoice_items) AS avg_num_invoice_utens, 
                          MIN(invoice_total) AS min_invoice_amt, 
                          MAX(invoice_total) AS max_invoice_amt,                    
                          AVG(invoice_total) AS avg_invoice_amt
                    FROM 
                    
                       customers AS c
                       
                           JOIN
                               -- Generate a table that contains:
                               --     Invoice Id
                               --     Total Line Items in the Invoice
                               --     Sum of the Unit Price of all items in the invoice
                               (SELECT 
                                    i.CustomerId,
                                    i.InvoiceId, 
                                    COUNT(ii.InvoiceLineId) AS num_invoice_items, 
                                    SUM(ii.UnitPrice * ii.Quantity) As invoice_total 
                               FROM 
                                    invoices AS i                             -- For the invoice ID
                                       JOIN invoice_items AS ii               -- For the invoice Line Items
                                    USING(InvoiceId)  
                               GROUP BY InvoiceId)  --end of new table
                               
                           USING(CustomerId) 
                           
                   GROUP BY usa_or_not
                   ORDER BY usa_or_not DESC
"""

results = pd.read_sql_query(sql_statement, conn)
results

,usa_or_not,min_num_invoice_items,max_num_invoice_items,avg_num_invoice_utens,min_invoice_amt,max_invoice_amt,avg_invoice_amt
0,USA,1,14,5.428571,0.99,23.86,5.747912
1,Other,1,14,5.439252,0.99,25.86,5.624735


---

You're done!  Now close the DB.

In [69]:
conn.close()